In [18]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
import pandas as pd
import numpy as np

In [1]:
# df = pd.read_csv("/home/jovyan/my_code/RAG/AD_pmid_results.csv")

In [2]:
from langchain_ollama import ChatOllama, OllamaEmbeddings
from itext2kg.graph_integration import GraphIntegrator

URI = "neo4j+s://d7f45b5f.databases.neo4j.io"
USERNAME = "neo4j"
PASSWORD = "uQK8FUAR8QUDyeKsMLcGh4DcLeq306tq-5CGCZWmZvk"

llm = ChatOllama(
    model="deepseek-r1:32b",
    temperature=0,
)

embeddings = OllamaEmbeddings(
    model="nomic-embed-text:latest",
)

/home/jovyan/miniconda3/envs/kg/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function array> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [3]:
def is_entity_in_context(entity, context):
    """
    Checks if an entity exists verbatim (case-insensitive) in a given context,
    returning "true" or "false" as strings.

    Args:
        entity (str): The entity to search for.
        context (str): The text context to search within.

    Returns:
        str: "true" if the entity is found in the context, "false" otherwise.
             Returns "false" if invalid input is detected.
    """

    if not isinstance(entity, str) or not isinstance(context, str):
        print(f"Error: Both entity {entity} and context must be strings.")
        return "false"  # Or raise a TypeError, depending on your needs

    entity_lower = entity.lower()
    context_lower = context.lower()

    if entity_lower in context_lower:
        return True
    else:
        return False

In [4]:
import json
from itext2kg.documents_distiller import DocumentsDistiller, DiseaseArticle

document_distiller = DocumentsDistiller(llm_model = llm)

IE_query = '''
# DIRECTIVES:
- As an experienced information extractor, your task is to extract biological entities from the provided bioinformatics context.
- Only extract entities that are explicitly mentioned in the context; do not generate or create any new terms.
- Extracted entities may include, but are not limited to, gene names, protein names, disease names, biological processes, pathways, molecular interactions, and other key bioinformatics terms.
- If an entity is not clearly mentioned in the context, leave it blank and do not infer or generate non-existent information.
- The output should only include the entities, excluding any non-entity content such as descriptive text or inferences.
'''

with open("/home/jovyan/my_code/RAG/Data_v2/AD_pubtabor/19295912.txt", "r") as f:
    text_line = f.readlines()
    PMID = text_line[0].split('|')[0]
    title = text_line[0].split('|')[-1]
    abstract = text_line[1].split('|')[-1]
context = f"Title: {title}Abstract: {abstract}."

distilled_1 = document_distiller.distill(documents=[context], IE_query=IE_query, output_data_structure=DiseaseArticle)

distilled_1
with open(f'/home/jovyan/my_code/itext2kg/datasets/demo_data/abstract/AD/19295912.json', 'w') as f:
    json.dump(distilled_1, f, indent=4)

In [5]:
distilled_1_ = {}
for key, value in distilled_1.items():
    if value != None and value != []:
        if isinstance(value, list):
            distilled_1_[key] = []
            for item in value:
                for k, v in item.items():
                    if is_entity_in_context(v, context):
                        distilled_1_[key].append({k: v})
        else:
            if is_entity_in_context(value, context):
                distilled_1_[key] = value

In [6]:
distilled_1_

{'disease': [{'disease': "Alzheimer's disease"}],
 'region': [{'region': 'prefrontal cortex'}],
 'processes': []}

In [7]:
pubtator_info = {}
pubtator_distilled = {
    'disease': [],
    'gene': [],
    'variant':[],
    'cell_line':[],
    'chemical':[],
}
species_info = []

seen_ids = set()
for entity_line in text_line[2:]:
    entity_line = entity_line.strip().split("\t")
    if len(entity_line) == 6:
        label = entity_line[4]
        name = entity_line[3]
        if label == 'Gene':
            unique_ID = f"Gene ID:{entity_line[5]}"
        else:
            unique_ID = entity_line[5]
        if name not in seen_ids:
            if label == 'Species':
                species_info.append(unique_ID)
                seen_ids.add(name)
            else:
                pubtator_distilled[label.lower()].append({label.lower():name})
                pubtator_info[name]={"label": label.lower(), "unique_id": unique_ID}
                seen_ids.add(name)


In [8]:
pubtator_distilled

{'disease': [{'disease': "Alzheimer's disease"},
  {'disease': 'AD'},
  {'disease': 'deficits in learning and memory'},
  {'disease': 'IAD'},
  {'disease': 'deficits in cognition'}],
 'gene': [{'gene': 'Abeta'}, {'gene': 'GluR2'}],
 'variant': [],
 'cell_line': [],
 'chemical': []}

In [9]:
def add_missing_entities(abstract_distilled, pubtator_distilled):
    miss_entities = []
    for entity_type, entities in pubtator_distilled.items():
        if entity_type not in abstract_distilled:  
            abstract_distilled[entity_type] = entities
            miss_entities.append({entity_type:entities})
            print(f'INFO --- {entities} added to the abstract')
        if isinstance(entities, list):  
            existing_names = set()
            if entity_type in abstract_distilled and isinstance(abstract_distilled[entity_type], list):
                existing_names = [item[entity_type] for item in abstract_distilled[entity_type]] #extract existing names, remove last character to match.  Use get for safety
            
            for entity in entities:
                entity_name = entity[entity_type]
                if entity_name and entity_name not in existing_names: 
                    print(f'INFO --- {entity_name} added to the abstract')
                    abstract_distilled[entity_type].append(entity) 
                    miss_entities.append({entity_type:entity})
                    
    return abstract_distilled, miss_entities

abstract_distilled, miss_entities= add_missing_entities(abstract_distilled = distilled_1_, pubtator_distilled = pubtator_distilled)
print(distilled_1_)
print(abstract_distilled)
print('miss_entities',miss_entities)

INFO --- AD added to the abstract
INFO --- deficits in learning and memory added to the abstract
INFO --- IAD added to the abstract
INFO --- deficits in cognition added to the abstract
INFO --- [{'gene': 'Abeta'}, {'gene': 'GluR2'}] added to the abstract
INFO --- [] added to the abstract
INFO --- [] added to the abstract
INFO --- [] added to the abstract
{'disease': [{'disease': "Alzheimer's disease"}, {'disease': 'AD'}, {'disease': 'deficits in learning and memory'}, {'disease': 'IAD'}, {'disease': 'deficits in cognition'}], 'region': [{'region': 'prefrontal cortex'}], 'processes': [], 'gene': [{'gene': 'Abeta'}, {'gene': 'GluR2'}], 'variant': [], 'cell_line': [], 'chemical': []}
{'disease': [{'disease': "Alzheimer's disease"}, {'disease': 'AD'}, {'disease': 'deficits in learning and memory'}, {'disease': 'IAD'}, {'disease': 'deficits in cognition'}], 'region': [{'region': 'prefrontal cortex'}], 'processes': [], 'gene': [{'gene': 'Abeta'}, {'gene': 'GluR2'}], 'variant': [], 'cell_line

In [10]:
abstract_distilled

{'disease': [{'disease': "Alzheimer's disease"},
  {'disease': 'AD'},
  {'disease': 'deficits in learning and memory'},
  {'disease': 'IAD'},
  {'disease': 'deficits in cognition'}],
 'region': [{'region': 'prefrontal cortex'}],
 'processes': [],
 'gene': [{'gene': 'Abeta'}, {'gene': 'GluR2'}],
 'variant': [],
 'cell_line': [],
 'chemical': []}

In [11]:
properties_info = {}
for k, v in abstract_distilled.items():
    if k not in ['disease', 'pathway', 'gene', 'metabolite', 'protein','processes', 'region','regulation'] and v != "" and v != None and v != []:
        properties_info[k] = v
properties_info['source'] = f'PMID{PMID}'
if species_info != []:
    properties_info['species'] = ','.join(list(set(species_info)))
print(properties_info)

semantic_blocks_1 = [f"{key} - {value}".replace("{", "[").replace("}", "]") for key, value in abstract_distilled.items() if value !=[] and value != ""  and value is not None]
semantic_blocks_1.append(f'{context}')
print(semantic_blocks_1)

{'source': 'PMID19295912', 'species': '9606'}
['disease - [[\'disease\': "Alzheimer\'s disease"], [\'disease\': \'AD\'], [\'disease\': \'deficits in learning and memory\'], [\'disease\': \'IAD\'], [\'disease\': \'deficits in cognition\']]', "region - [['region': 'prefrontal cortex']]", "gene - [['gene': 'Abeta'], ['gene': 'GluR2']]", "Title: Transcriptome analysis of synaptoneurosomes identifies neuroplasticity genes overexpressed in incipient Alzheimer's disease.\nAbstract: In Alzheimer's disease (AD), early deficits in learning and memory are a consequence of synaptic modification induced by toxic beta-amyloid oligomers (oAbeta). To identify immediate molecular targets downstream of oAbeta binding, we prepared synaptoneurosomes from prefrontal cortex of control and incipient AD (IAD) patients, and isolated mRNAs for comparison of gene expression. This novel approach concentrates synaptic mRNA, thereby increasing the ratio of synaptic to somal mRNA and allowing discrimination of expre

In [12]:
semantic_blocks_1

['disease - [[\'disease\': "Alzheimer\'s disease"], [\'disease\': \'AD\'], [\'disease\': \'deficits in learning and memory\'], [\'disease\': \'IAD\'], [\'disease\': \'deficits in cognition\']]',
 "region - [['region': 'prefrontal cortex']]",
 "gene - [['gene': 'Abeta'], ['gene': 'GluR2']]",
 "Title: Transcriptome analysis of synaptoneurosomes identifies neuroplasticity genes overexpressed in incipient Alzheimer's disease.\nAbstract: In Alzheimer's disease (AD), early deficits in learning and memory are a consequence of synaptic modification induced by toxic beta-amyloid oligomers (oAbeta). To identify immediate molecular targets downstream of oAbeta binding, we prepared synaptoneurosomes from prefrontal cortex of control and incipient AD (IAD) patients, and isolated mRNAs for comparison of gene expression. This novel approach concentrates synaptic mRNA, thereby increasing the ratio of synaptic to somal mRNA and allowing discrimination of expression changes in synaptically localized gen

## iText2KG for graph construction

In [13]:
from itext2kg import iText2KG

itext2kg = iText2KG(llm_model = llm, embeddings_model = embeddings)

In [14]:
kg1 = itext2kg.build_graph(
    sections=[semantic_blocks_1], 
    source=properties_info,  
    entities_info=pubtator_info,
    ent_threshold=0.9, 
    rel_threshold=0.4
    )

[INFO] ------- Extracting Entities from the Document 1
INFO --- alzheimer's disease add unique_id and label
INFO --- ad add unique_id and label
INFO --- deficits in learning and memory add unique_id and label
INFO --- iad add unique_id and label
INFO --- deficits in cognition add unique_id and label
INFO --- abeta add unique_id and label
INFO --- glur2 add unique_id and label
{'entities': [{'label': 'disease', 'name': "Alzheimer's disease", 'properties': {'unique_id': 'MESH:D000544'}}, {'label': 'disease', 'name': 'AD', 'properties': {'unique_id': 'MESH:D000544'}}, {'label': 'disease', 'name': 'deficits in learning and memory', 'properties': {'unique_id': 'MESH:D007859'}}, {'label': 'disease', 'name': 'IAD', 'properties': {'unique_id': 'MESH:D000544'}}, {'label': 'disease', 'name': 'deficits in cognition', 'properties': {'unique_id': 'MESH:D003072'}}, {'label': 'Region', 'name': 'prefrontal cortex', 'properties': {}}, {'label': 'gene', 'name': 'Abeta', 'properties': {'unique_id': 'Gene

In [22]:
kg1.relationships[0].startEntity

Entity(name=alzheimer's disease, label=disease, properties=embeddings=array([ 7.06933796e-02,  1.94239926e-02, -1.58417734e-01, -3.11379006e-02,
       -5.82284680e-03,  3.57061112e-02, -9.69454200e-03, -6.11782662e-03,
        7.67751640e-02,  1.58176158e-02,  3.63132770e-02,  7.23955896e-02,
        9.75482156e-02, -9.10408470e-03,  1.83855084e-02, -9.82803040e-03,
       -8.11118310e-02, -1.09822160e-02, -4.78328164e-03,  3.16027808e-02,
       -9.48155980e-03,  1.30063704e-02, -1.08970132e-02, -1.08200678e-03,
        1.06816304e-01,  7.87549648e-02, -7.00410400e-02,  1.66296412e-02,
       -4.33805200e-03,  3.35680628e-02,  7.17193296e-02, -2.93132580e-03,
       -1.02235556e-02, -2.83019018e-02,  6.26340450e-03, -2.55154994e-02,
        5.38056136e-02,  2.65617242e-02,  7.16301240e-02,  1.51285612e-02,
        4.81696240e-04,  2.10209824e-02,  9.68531720e-03, -2.32958214e-02,
        2.58304004e-02,  6.65344540e-02,  1.28155042e-02,  1.01749120e-02,
        2.86977392e-02, -1.124

In [ ]:
unique_rj = []
# for ri in kg1.relationships:
#     for rj in kg1.relationships:
#         if ri == rj:
#             continue
#         else:
#             if ri.startEntity == rj.startEntity and ri.endEntity == rj.endEntity:
#                 unique_rj.append(rj)
for ri in kg1.relationships:
    for rj in kg1.relationships:
        if ri == rj:
            pass
        else:
            if ri.startEntity == rj.startEntity and ri.endEntity == rj.endEntity:
                       
                print(ri.startEntity.name,'-', ri.endEntity.name,'-',ri.name)
                print(rj.startEntity.name,'-', rj.endEntity.name, '-',rj.name)
        unique_rj.append(ri)

alzheimer's disease - deficits in learning and memory - causes
alzheimer's disease - deficits in learning and memory - impacts
alzheimer's disease - prefrontal cortex - associated_with_region
alzheimer's disease - prefrontal cortex - impacts
alzheimer's disease - glur2 - increases_expression_of
alzheimer's disease - glur2 - impacts
alzheimer's disease - prefrontal cortex - impacts
alzheimer's disease - prefrontal cortex - associated_with_region
alzheimer's disease - deficits in cognition - impacts
alzheimer's disease - deficits in cognition - causes
alzheimer's disease - deficits in learning and memory - impacts
alzheimer's disease - deficits in learning and memory - causes
alzheimer's disease - glur2 - impacts
alzheimer's disease - glur2 - increases_expression_of
alzheimer's disease - deficits in cognition - causes
alzheimer's disease - deficits in cognition - impacts


In [16]:
GraphIntegrator(uri=URI, username=USERNAME, password=PASSWORD).visualize_graph(knowledge_graph=kg1)

In [17]:
import numpy as np
from numpy.linalg import norm

a = np.array([0.011370918,-0.018428847,-0.108812936,-0.013682617,0.039464086,-0.015505651,-0.029547973,0.006033173,-0.02580918,-0.008745876,-0.018480806,0.06577632,0.06762227,-0.01570069,-0.013936541,-0.06868806,-0.052637257,-0.092928395,0.051473413,0.013683683,0.033443436,-0.045841776,-0.005812288,0.01690823,0.17991394,0.008434932,0.013810512,0.019580701,-0.029805895,-0.037383664,0.014473434,-0.062506475,-0.005614584,-0.008359527,-0.03193108,0.004041477,0.03156871,0.044183407,0.029748341,0.010578768,0.00065839203,0.027094588,-0.04909457,0.015211493,0.0533151,0.03686782,0.011273931,0.003926505,0.05186136,-0.030411264,-0.0138438335,0.0137209855,-0.037532873,-0.049550727,0.087207235,0.043130405,0.05350694,0.017943647,-0.0109668495,0.044191934,0.0051189912,0.036390346,-0.016872793,0.054197572,0.038018875,-0.06739206,-0.049401518,0.00736621,-0.024525968,-0.04983636,0.06026405,-0.039459825,0.07264427,0.046212673,-0.035806295,-0.01358563,-0.0005662761,-0.025772944,-0.018603005,0.024741258,0.03951098,-0.03069796,0.068133846,-0.010251038,-0.0004327109,0.011865444,-0.009000666,-0.019810911,0.03383768,0.014971158,0.004331705,-0.0070800455,0.05041615,0.04061514,-0.089637235,-0.0032826343,-0.025403112,0.021275308,-0.013113485,-0.021468949,-0.018609216,0.027759576,0.022473257,-0.025726048,0.04645354,-0.027043365,-0.013013034,-0.021760775,-0.022778604,0.00033625687,-0.012612564,0.06647548,0.032504477,-0.027829919,-0.023845995,-0.017379845,0.053698782,-0.07651523,0.027970603,-0.05223652,-0.022946468,-0.025195286,0.022873994,0.06392611,-0.02117619,0.03828319,-0.055621468,0.040099297,0.02374581,-0.01773795,-0.0188906,-0.046724252,0.0074210986,-0.023573153,0.01958443,0.027442502,-0.045675512,-0.051330596,0.01062966,0.027378025,0.046289407,0.042437643,0.0012419123,-0.04935462,-0.018080069,-0.069127165,0.03739219,-0.026642628,-0.0071362657,0.029223973,0.01416915,0.09067745,0.028085709,0.015926056,0.0016860804,-0.020481294,0.016615672,0.01955139,-0.006986789,-0.045487933,0.05915563,0.018881543,-0.02483718,0.062029,-0.01789995,-0.08811102,-0.008271866,0.048007462,-0.0022711994,0.037733242,0.004541333,-0.008734153,0.017748607,-0.025649842,0.026651155,-0.03273682,-0.010345628,-0.0008233231,0.038219243,-0.019405112,0.0048357244,-0.03169554,0.0240826,0.023305103,0.004785399,-0.028411841,0.030498657,-0.030402737,-0.04286609,-0.030141084,0.024854232,-0.0014117724,-0.013912294,-0.045394145,-0.0054030246,-0.003973666,0.008408819,0.021556543,0.068304375,0.012165997,0.032408554,0.011285654,-0.061935212,0.0048632016,-0.037481718,0.06754553,0.023363989,-0.02000382,-0.017498147,0.045502853,0.07889407,-0.021033373,-0.03153674,-0.006980927,-0.009662089,-0.032203924,0.009063481,-0.052824836,-0.044571355,0.044528723,0.005726492,0.015973,0.0007315317,0.003874681,-0.0015709751,-0.009947821,-0.025340233,0.009911851,-0.06294132,0.016818704,-0.039344717,-0.086414285,0.03408824,0.014082821,0.007372072,0.04367609,0.0061394856,0.029466974,0.04196017,-0.0028685748,-0.009017956,0.051383886,0.012001866,-0.016279414,-0.0392424,0.0070877722,-0.038070034,-0.02479668,0.045905724,0.032775186,-0.012468415,-0.017135244,0.017047318,0.02738868,0.032883897,-0.002130515,-0.043011036,-0.013850478,0.026290918,-0.0134353535,0.07758102,-0.059445526,0.013159314,-0.06103142,0.006142416,0.004285277,0.0013921221,-0.027413728,0.04801599,0.009657393,-0.015867487,0.060477212,0.0112030385,0.03702449,0.037036218,0.019325977,0.013945867,-0.022353888,0.004218665,-0.005858117,0.03873935,0.013563248,-0.029611921,0.0042503723,-0.05733526,0.043211404,0.04295988,0.011928326,0.018037437,0.008164087,0.006391012,-0.031383265,-0.068509005,0.039295692,-0.02016049,0.010418667,0.11094452,-0.017678797,-0.0015100918,-0.025206475,0.005903413,0.058746368,0.016667897,-0.007874859,0.030575395,-0.006832782,0.033001132,-0.066692896,0.026606392,-0.0033231345,-0.033111975,0.023918469,0.0014545375,0.061952263,-0.04981717,0.036436174,0.0010140996,-0.028102761,0.03214104,-0.0013511558,-0.025141995,-0.0329244,-0.003204299,-0.008942247,-0.032289185,0.04110114,0.0063952752,0.028831761,-0.014984879,-0.075440906,-0.038193665,0.018877279,-0.029652951,-0.032969162,0.026920265,0.019785864,-0.014131847,-0.024736995,-0.022496704,0.085425235,0.025817707,-0.05338757,0.043681417,-0.042179722,-0.023202257,0.0025527012,-0.05059094,-0.023304572,0.026568156,0.04246535,-0.0014730887,0.03778227,-0.023242757,-0.0076054805,0.009886271,-0.011595,0.064424895,0.014348736,0.04057677,-0.03708098,-0.0044507408,-0.047278464,0.011940315,-0.020445056,0.019913228,0.02081915,-0.009537025,0.02177623,0.036910452,-0.0046722917,0.008395497,-0.024762575,0.022692809,0.027209628,0.06392185,-0.0054540904,-0.12271085,-0.006250594,-0.042930037,0.016984437,0.021369098,-0.018199436,0.009369363,-0.022843083,0.057923578,0.007184493,-0.012732998,-0.0035963764,0.012320537,-0.046513222,-0.0060731396,0.025448943,-0.04684362,0.0016148055,-0.060443103,0.032192197,0.030087262,-0.006667052,-0.0031755224,0.006838377,0.032003287,-0.019873526,0.044515934,-0.002293581,0.02240611,0.02798126,0.0007798422,-0.07868091,0.061982106,-0.02032036,-0.003317139,0.0020804228,-0.016622066,-0.059876103,-0.039432112,0.03590861,0.03234354,-0.018470014,-0.054308414,-0.006265782,0.05187415,0.090737134,-0.007919355,0.04146564,-0.012703156,0.032668605,-0.0063970736,0.06719169,-0.017972223,-0.04941857,-0.033218555,-0.0011228102,0.029509604,-0.031905502,-0.0026873904,0.002839532,-0.012883807,0.0019785066,0.04770478,-0.0130981635,0.03759682,0.0009805271,-0.036707953,0.018477341,0.022292072,0.050400157,0.07449449,-0.0018981727,-0.038952507,0.00089313235,0.02205973,0.018239137,-0.0069009927,-0.031157317,0.10073851,-0.037601084,0.00037089505,0.025208075,-0.020891624,0.034791663,0.02668526,0.008876435,-0.02284948,0.024017585,-0.023202257,-0.06670995,0.028882919,-0.051571466,-0.0030548216,0.078322805,-0.011183604,0.028094234,-0.041160826,-0.033657663,0.012231277,0.04579062,-0.024564337,-0.0036914318,0.0072335196,0.026487023,0.04712925,-0.014475565,-0.06734943,-0.04770904,0.0041304706,0.027618892,-0.009042133,-0.015541355,0.0119336555,0.028985236,0.04139317,0.026962366,0.025017297,-0.03772472,-0.03169021,-0.04664325,-0.062651426,0.038334347,0.012248329,0.02338344,0.011180873,-0.026655683,-0.010118614,0.016387593,0.014954638,0.016074251,0.030846106,-0.012021783,-0.049171306,-0.04119067,-0.03815743,0.018527431,0.015597843,0.034238514,0.061948,-0.025589628,0.07992575,0.03793361,-0.07906459,0.062208053,0.013023959,-0.013586696,-0.026110798,0.02510789,-0.04112672,0.034983505,0.04295988,-0.08066754,0.025187824,-0.011550503,0.020119991,-0.006226081,-0.031937476,-0.031023027,0.015928837,-0.02061505,-0.0028802985,0.00535986,0.022791393,0.015701225,-0.01182601,-0.0013365012,-0.023673337,0.010506562,0.007189289,-0.018187312,-0.09598935,0.03515403,0.0049703135,-0.022710927,-0.016027356,-0.05567689,-0.021220952,-0.0074661276,-0.020390702,-0.053924732,-0.02670764,0.0011470569,0.019680684,-0.056533787,0.024577126,0.006487466,0.0036652533,-0.023468705,-0.035335217,-0.016121145,0.027946088,-0.0003996714,0.039297823,-0.005014144,-0.026512602,-0.031803187,-0.026975155,-0.015538957,0.019069122,-0.020668872,0.008244421,0.0037459868,-0.013889381,-0.004448942,0.055587362,-0.105692305,-0.01754504,-0.029157894,-0.0162946,-0.025240049,-0.0024646402,-0.014050847,0.03927544,-0.0146365,0.0012925373,0.0031999024,-0.024895797,0.03569332,0.009121034,0.030302553,-0.07364185,-0.023971757,0.01718587,0.018652398,0.057122104,0.020838333,0.0728148,-0.040598087,-0.021111175,0.07936302,0.017531186,0.04724862,-0.030938294,0.018911384,0.019881787,0.02759278,-0.0052058534,-0.015671382,0.018943358,-0.05243689,0.004554122,-0.03148984,-0.041659616,-0.0091871135,0.0014350867,-0.05313178,0.03872656,0.007971846,0.068815954,0.0051690834,-0.04302383,0.028306326,-0.0156030385,0.030417658,-0.047836937,0.024423653,-0.0698988,-0.032946512,-0.062003423,0.03906761,-0.0040774476,0.06098879,-0.00048137092,0.070359215,0.022590494,0.028435288,-0.0011835602,0.06589995,0.026856318,0.004393188,-0.004842152,-0.006589515,0.015134224,-0.0063129426,0.09401124,0.0754878,0.020114662,0.02218869,-0.012982392,-0.0054131495,-0.016374804,0.022852678,-0.087514184,-0.05040336,-0.057220154,-0.04405125,0.013961854,0.027656194,0.029096078,-0.028156051,0.021592913,-0.029724894,-0.022464197,-0.034593426,-0.026068166,0.014634101,0.031058995,-0.030230079,0.0016414504,0.07147617,0.033934765,0.027814997,0.0006771432,-0.065631375,-0.0015123566,0.00556236,-0.0001662633,-0.003468082,0.026938919,-0.025583765,0.020565491,-0.00728148,-0.033570267,-0.054423522,0.015836712,-0.047201727,0.037063926,-0.060835317,0.00039913852,-0.019336635,-0.027177654,-0.00085043436,-0.056827944,0.03432578,-0.022960322,0.053408887,0.03427582,0.0802327,-0.037481718,-0.02293794,-0.012304749,0.013086307,-0.013993828,-0.0500282,-0.03198437,0.017504541,0.020900149,0.014624775,0.033073608,-0.0262712,-0.04329667,-0.036603507,-0.009308613,-0.025864601,0.03167103,-0.006213291,-0.08552755,-0.012636011,-0.030430447,-0.022260098,0.09407093,-0.03829172,0.02098328,-0.05395457,0.0015455295,0.039059088,-0.03450603,0.04472909,-0.03161987,-0.0439873,0.0019491974,-0.053106204,-0.051520307,0.004789662,-0.042090196,-0.009938229,0.021403203,0.01998357,0.024203831,-0.00078662,-0.0058450606,0.03188818,0.03322495,-0.012927237,-0.019718189,0.02468264,0.004015365,0.004923552,-0.018199436,0.047355197,-0.047939252,0.019256702,0.054571666,0.015821658,0.014244288,0.01943682,-0.0010063726,0.0115259895,-0.033367764,-0.06595963,-0.027908787,-0.051511783])
b = np.array([-0.041708693,-0.009128049,-0.11018078,-0.000471094,0.06121505,-0.00069628784,0.008505164,-0.06378832,-0.0060326327,-0.028661797,-0.0678726,0.053779215,0.0683564,0.036621753,0.008596699,-0.017204134,-0.016324366,-0.05891874,-0.022564307,0.043983966,-0.055696893,-0.046504647,0.028714383,0.0342378,0.09741304,0.036797043,0.03820989,0.027468944,-0.085485816,0.012530333,0.016824411,0.02118214,-0.03628169,-0.00086045853,-0.08198701,-0.033281587,0.056275357,0.00575633,0.0026346208,0.008737588,-0.030597007,0.040274814,0.039703365,-0.030914284,0.060314056,-0.03226972,0.031096587,0.025434695,0.056264836,-0.0043288064,-0.0046270196,0.0021524613,-0.02876522,-0.026716068,0.11636504,0.035754062,0.0913265,0.022390768,-0.0059767584,-0.01011888,0.034032706,0.027477706,-0.012605161,0.058320995,0.052541662,-0.022546778,-0.073864035,0.015408937,-0.020530276,-0.013993025,0.041414205,0.0127925025,0.013886098,0.01472837,-0.017115394,0.010615829,0.03847808,-0.0529273,-0.0030713137,0.024969297,-0.0038783082,-0.015674064,0.07109094,-0.070971735,0.046159323,-0.015694661,0.004017713,-0.044969097,-0.050340008,0.01524608,0.04429248,0.024552105,0.030921297,0.029710036,-0.08165045,0.010964439,0.02041524,0.03117021,-0.03132271,-0.05395801,-0.031708352,-0.005956381,0.028021986,-0.030234156,0.05615616,0.01581123,-0.012047736,-0.01009401,0.015223129,0.009060781,-0.031215785,0.021751834,-0.002125839,-0.043061938,0.049740516,-0.056450646,-0.0018501943,-0.04743018,0.027016694,0.018361272,0.007846454,-0.03434823,-0.024330553,0.057179857,-0.011105767,-0.015765872,-0.02468445,-0.008271096,0.013924662,-0.051058702,-0.020126011,-0.053116616,-0.022758441,0.002414302,-0.014782711,-0.012344854,-0.0068665785,-0.0556373,0.011866747,0.011807368,0.042378303,0.0341186,-0.02204435,-0.04636266,-0.0039256373,-0.07265803,0.049645856,-0.045256574,-0.022942934,-0.04463079,-0.00808967,0.012021881,-0.017813379,0.044225864,-0.017152643,-0.024287414,-0.027634593,-0.0037368706,0.01446894,-0.016373036,0.03588027,-0.0148041835,-0.016179778,0.020829806,-0.024472347,-0.047766738,0.041168686,0.0020149676,0.039805032,0.073762365,-0.039005708,-0.04649413,-0.035889037,-0.027317317,0.01880443,-0.004999074,-0.06301354,-0.006093984,0.01727929,-0.041277476,-0.017520314,-0.10587564,0.021244368,-0.00022632386,-0.008150145,0.022963613,0.04331085,-0.010270068,-0.040162627,-0.033319276,-0.015844509,-0.021736935,-0.012044339,-0.050501276,-0.028192017,-0.0196444,0.045168933,-0.010579457,0.060019568,-0.072514296,-0.028370814,0.0061959843,-0.023274234,0.032379717,-0.028206041,0.031929217,0.0006492333,0.0052964115,-0.044749986,0.02891071,0.10046266,-0.0024720384,-0.019262705,0.018913876,-0.05796516,-0.04928476,-0.013810722,-0.032649662,-0.0007353449,0.036295712,0.026368117,0.0060785366,-0.047202304,-0.006222823,0.017438805,-0.040734075,-0.01896997,0.045011166,-0.05212447,0.04923918,-0.038153794,-0.0072822366,0.0418051,0.052524135,0.013619218,0.037736602,0.031087821,0.00938726,-0.0059837704,0.0326707,-0.0007317295,0.04824353,-0.017710505,-0.00014943541,-0.05138474,0.041549176,-0.030095676,0.03376539,-0.013222185,0.0588346,0.00081247283,0.020458188,-0.013853231,0.010096312,0.023874607,-0.041540414,-0.03132797,0.026603883,0.0060805636,-0.047812317,0.021269565,-0.029047437,0.009676052,-0.008190681,0.00047372337,-0.052517124,-0.021849995,0.013453568,0.011072024,-0.040593844,-0.01880432,0.034077406,-0.00083449367,0.015517617,0.018444974,0.007470455,0.0032374016,-0.053421624,0.002153228,-0.0035337529,-0.025271673,-0.019946283,-0.021197915,0.007802796,0.010700626,0.008945089,0.04368597,0.029655697,-0.0067151706,-0.04734604,0.02937523,-0.008685659,-0.095189914,0.03349867,-0.0042380933,0.01896997,0.0651591,-0.013578901,-0.018988814,0.0036092375,0.012667389,0.026615057,0.007253533,0.012336967,-0.015338383,-0.07590091,0.02519805,-0.021616861,0.017398488,-0.010225807,-0.056443635,0.011074873,-0.014452726,0.043135557,-0.05770222,0.03362077,0.03865162,0.012558707,-0.004620227,-0.011534134,0.016235434,-0.0061158957,0.061288673,-0.080886185,-0.054785382,0.059016906,0.003743773,0.021504674,-0.014486907,-0.02514371,-0.047829844,0.04076563,0.030419964,-0.023999061,0.014042546,0.0061314525,0.005233307,0.0042681117,-0.011956146,0.06651234,0.047360066,-0.011728268,0.0070966478,-0.041614033,-0.0069257393,0.010709829,-0.01944824,0.0099534495,0.034349985,0.03574705,-0.052503098,0.045912165,-0.030763535,-0.018565267,0.030288495,-0.026174422,0.05022432,0.048678253,-0.033866618,-0.014677974,0.01649837,-0.020060278,-0.05026989,-0.03775413,0.014459738,0.02461784,0.01558861,0.0027546948,0.06949929,0.017409882,0.0040010125,-0.003285607,0.0039094226,-0.0042403936,0.0686018,0.019337203,-0.05993543,0.005115862,0.020027412,-0.012994416,0.033301745,-0.008109828,-0.0062981984,0.015884852,0.07248625,-0.008674265,0.022389892,-0.015677132,0.026042951,0.054033387,-0.043412514,0.010023127,-0.0064726127,0.013081731,-0.009857204,-0.0015862173,0.039812047,0.01856932,0.027963523,0.008046285,0.007459061,-0.03599947,0.018752608,-0.027106969,0.020050637,0.022579206,-0.0057328846,-0.053642485,0.03654112,-0.03312996,0.015008398,0.04272187,-0.011024476,-0.07682645,-0.04923568,-0.0028729613,0.031378806,-0.0051544807,-0.040818214,-0.008315356,0.003587764,0.07112599,0.015574148,0.066203825,-0.0054200185,-0.027126249,0.025785277,0.042392325,-0.03467252,-0.068188116,-0.073502935,0.02317695,0.0341186,-0.015205162,-0.03566291,0.016314533,0.02790454,0.011375276,0.035019595,0.05198073,0.048264567,-0.0014838366,-0.07401478,-0.004777448,0.05283966,0.10251006,0.016751664,-0.033547148,-0.053642485,0.023371082,-0.020624276,-0.0020360574,0.008276246,0.007361775,0.08390119,-0.007512087,0.020145621,0.023119101,0.0095077725,-0.03056874,0.009865804,0.012750653,-0.05121997,0.0015062956,0.011909694,-0.06295744,0.048962224,-0.023838669,0.005252424,0.07207958,-0.013907571,0.031561106,0.06833536,0.014084616,0.0047644037,-0.017117584,-0.04329332,-0.011599868,0.0066015613,0.032344658,0.016461559,-0.019745631,-0.056475185,-0.060892515,-0.004964715,0.02215774,0.027115732,-0.010956332,0.045558076,0.022668606,0.005209697,0.05100261,0.030059302,-0.021769363,-0.035690956,0.014849293,-0.050375067,0.027236683,0.03736849,-0.0042657014,0.043864768,0.0012235298,-0.06627394,0.0051938663,-0.040034663,0.00266442,0.020690996,-0.068552725,-0.06913469,0.03617301,-0.02794661,0.010482827,0.027937846,0.060650617,-0.020132149,-0.0003426935,0.061148442,-0.03896539,-0.058336776,0.05978117,0.037694532,-0.025042698,-0.06656142,-0.035717253,-0.030536968,0.016259974,0.038244944,-0.052527636,0.018649187,0.03439556,0.015839932,0.005826446,-0.026376005,0.0024818983,0.024738789,-0.057425264,-0.016286267,0.01604787,0.040534243,0.048531007,0.025189288,0.0064765564,-0.047500297,0.005799496,0.035778604,-0.00055213866,-0.022399094,0.035314083,0.011526028,-0.0076591116,0.036761984,-0.051717795,0.010758473,0.0049747527,0.016284514,-0.030416459,0.01340098,0.01473582,-0.0011076187,-0.056173686,0.060994186,0.0047455598,0.01803041,0.005481069,0.009538448,-0.03239549,-0.00787954,0.028872145,0.017494896,0.006083467,0.03658494,-0.050630994,-0.008728605,-0.072598435,-0.0033656927,-0.00517557,0.0055041304,-0.007029161,-0.049263727,-0.017322237,0.07048092,-0.06541853,0.005322705,-0.011175884,-0.015533831,0.015905887,0.012998908,-0.0019774446,0.041603517,-0.028970309,0.003282375,0.0072351275,0.006324492,-0.0222146,0.052713446,0.000604315,-0.044117186,-0.00071518647,-0.021272413,0.029343022,0.011102919,-0.010127644,0.07083151,-0.0076070726,-0.049673904,0.03403621,-0.042651758,0.01672274,-0.019706191,0.0025075348,-0.0017336258,0.053081557,0.014039916,0.015923634,0.07695967,-0.031588934,0.011018779,-0.037095036,-0.059409555,-0.047570415,-0.058042288,-0.07810958,0.016689766,-0.034991547,0.0358382,-0.043296825,-0.06045429,-0.06416695,-0.0037122208,0.015713943,-0.02859694,0.04019418,-0.06001606,0.0036960065,-0.055689882,0.013398022,-0.015736496,0.012707706,-0.0012487277,0.07443548,0.056731112,0.028202536,0.0063973465,0.066659585,0.04649062,-0.050077073,0.011688115,0.03346827,0.0062182215,-0.0014599533,0.1005468,0.0762515,0.04752484,-0.007997396,0.032921363,-0.050988585,-0.037466656,-0.03874628,-0.041138995,0.02126146,-0.055202577,-0.05296587,-0.06905055,0.040593844,0.003178022,-0.012170768,0.033051077,0.029189423,-0.035338625,0.008700559,-0.02526006,-0.04077176,-0.008467202,-0.028301902,-0.033711925,0.0085734725,0.033158008,0.019810269,0.016954564,-0.0026994783,0.033119444,-0.0026937814,0.036428932,0.03119475,0.017810421,-0.018594848,0.05680123,-0.0059228563,-0.051538996,-0.053358518,-0.010351085,-0.06879112,-0.02988708,-0.014788846,-0.0059995465,-0.04905688,-0.010964,0.030627025,-0.063294,0.09991575,-0.05022782,0.031548508,0.026321666,0.027938148,-0.014330898,0.00021319743,0.008773167,-0.010760115,-0.017559756,-0.0021957362,-0.027641604,0.037317656,0.039570145,0.04988776,0.01977806,0.031401593,0.0201488,0.015725337,0.02508499,0.050536335,0.014692435,0.018225422,-0.09622062,0.0037261343,-0.039047778,-0.011636679,0.05761458,-0.009175048,0.021834876,0.011507401,0.0114166895,-0.019971756,-0.06485058,0.04514965,0.012392182,0.009019807,-0.007773708,-0.03554547,-0.07279476,-0.025518835,-0.015143152,-0.030276226,0.013021914,0.071161054,-0.016472952,-0.020468706,0.025807297,0.022275954,0.01971846,-0.0039602574,-0.00031355125,0.0056992513,-0.0012669142,0.0031280643,0.0050974563,-0.021655425,-0.05860322,0.051819462,0.067648225,0.012792106,0.05502378,0.00020991072,-0.060019568,0.02824811,-0.0161342,-0.0036873512,-0.044972606,-0.0030895122])

# 计算余弦相似性
cos_sim = np.dot(a, b) / (norm(a) * norm(b))
print(cos_sim)

0.6004619832921774
